In [ ]:
import pandas as pd
from transformers import pipeline
from huggingface_hub import notebook_login

notebook_login()

Import loading and login process into hugging face for model authorization.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
def format_prompt(text, task):
    if task == "orientation":
        return f"Bu konuşmanın sol görüşlü bir siyasetçiye (class 0) mi yoksa sağ görüşlü bir siyasetçiye (class 1) mi ait olduğunu sınıflandır: {text}"
    elif task == "power":
        return f"Bu konuşmanın iktidar partisinden (class 0) mi yoksa muhalefet partisinden (class 1) mi yapıldığını belirle: {text}"
    else:
        raise ValueError("Invalid task. Choose 'orientation' or 'power'.")

model_name = "meta-llama/Llama-3.2-1B"
classifier = pipeline("text-classification", model=model_name, tokenizer=model_name, device=0)

test_data_path = "/kaggle/input/power-dataset/power-tr-train.tsv"
task = "power"

print("Loading test data...")
test_data = pd.read_csv(test_data_path, sep='\t')
test_data = test_data.dropna(subset=["text", "label"])

test_data = test_data.head(500)

print("Running zero-shot inference...")
predictions = []
for index, row in test_data.iterrows():
    prompt = format_prompt(row['text'], task)
    result = classifier(prompt)
    predicted_label = result[0]['label']
    predictions.append(predicted_label)

true_labels = test_data['label'].tolist()
predicted_labels = [int(label[-1]) for label in predictions]

accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(
    true_labels, predicted_labels, average='binary', zero_division=0
)

print(f"Zero-shot inference results for {task} task (Turkish text):")
print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1 Score: {f1:.2%}")


### Zero-Shot Inference Flow - Turkish
- Inference flow with Turkish prompts.
- File path and task type can be changed to variate between power and orientation tasks.